In [1]:
import hoomd
import hoomd.md

# Advancing simulations forward in time

Use `hoomd.run` to advance a simulation forward a given number of time steps. Start with the Lennard-Jones tutorial script.

In [2]:
hoomd.context.initialize('--mode=cpu');
system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=2.0), n=5);
nl = hoomd.md.nlist.cell();
lj = hoomd.md.pair.lj(r_cut=3.0, nlist=nl);
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0);
all = hoomd.group.all();
hoomd.md.integrate.mode_standard(dt=0.005);
hoomd.md.integrate.langevin(group=all, kT=0.1, seed=987);

HOOMD-blue v2.1.5 CUDA (7.5) DOUBLE HPMC_MIXED MPI SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX 
Compiled: 03/09/2017
Copyright 2009-2016 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 125 particles
notice(2): integrate.langevin/bd is using specified gamma values


After initialization the simulation starts at time step 0. Query the current step counter with ``get_step``.

In [3]:
hoomd.get_step()

0

Each HOOMD script can have one **integrator** active at a time. This example uses the `mode_standard` integrator from the `md` package. On each time step, `mode_standard` advances the simulation forward `dt` time units, integrating the equations of motion specified in the provided integration methods (Langevin dynamics in this example). Use `hoomd.run` to advance the simulation a given number of steps.

In [4]:
hoomd.run(1000);

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 125
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:00 | Step 1000 / 1000 | TPS 1781.45 | ETA 00:00:00
Average TPS: 1759.46
---------
-- Neighborlist stats:
32 normal updates / 11 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 21 / n_neigh_avg: 9.208
shortest rebuild period: 19
-- Cell list stats:
Dimension: 2, 2, 2
n_min    : 9 / n_max: 21 / n_avg: 15.625
** run complete **


Now we are at step 1000.

In [5]:
hoomd.get_step()

1000

## Run options

Supress run output with the `quiet` option.

In [6]:
hoomd.run(1000, quiet=True);
hoomd.get_step()

2000

Use `limit_hours` to stop a run after so many hours, even if it has not reached the requested number of time steps.

In [7]:
hoomd.run(1e6, limit_hours=5/3600, quiet=True);

notice(2): Ending run at time step 60150 as 0.00138889 hours have passed


The profile option prints information on how much time was spent in each portion of the calculation.

In [8]:
hoomd.run(10000, profile=True)

** starting run **
Time 00:00:06 | Step 70150 / 70150 | TPS 10854.1 | ETA 00:00:00
Average TPS: 10800.2
Simulation:  0.9262s | 100.000% 
        Integrate:  0.0994s | 10.733% 
                Langevin step 1:  0.0273s |  2.945% 
                Langevin step 2:  0.0432s |  4.665% 
                Net force:        0.0198s |  2.141% 
                Sum accel:        0.0000s |  0.000% 
                Self:             0.0091s |  0.982% 
        Neighbor:   0.0721s |  7.783% 
                Cell:        0.0018s |  0.196% 
                        compute:  0.0016s |  0.176% 
                        init:     0.0000s |  0.002% 
                Dist check:  0.0118s |  1.272% 
                compute:     0.0550s |  5.943% 
                head-list:   0.0000s |  0.005% 
                Self:        0.0034s |  0.367% 
        Pair lj:    0.7409s | 79.991% 
        SFCPack:    0.0024s |  0.263% 
        Self:       0.0114s |  1.229% 
---------
-- Neighborlist stats:
141 normal updates / 101

## Callbacks

Specify a callback and this function will be called at the given period. If the function returns a negative value, the run is aborted.

In [9]:
def my_callback(step):
    print("This is step", step)
    if step >= 70000:
        return -1
    else:
        return 0

hoomd.run(1e6, callback=my_callback, callback_period=1000, quiet=True)

This is step 71000
notice(2): End of run requested by python callback at step 71000 / 1070150
